# 1. Prepare

In [6]:
import sys
#rootPath = '/Users/floyd/Desktop/Research/NER-CRF/cctner/'
rootPath = '/home/floyd/Desktop/cctner/'


sys.path.append(rootPath)

import pickle
import random

# 2. Load Data

In [7]:
CCT_DictPath = rootPath+ 'pkldata/ccks/CCT_Dict.p'
print(CCT_DictPath)
with open(CCT_DictPath, 'rb') as handle:
    CCTDict = pickle.load(handle)
[(k, len(CCTDict[k])) for k in CCTDict]

/home/floyd/Desktop/cctner/pkldata/ccks/CCT_Dict.p


[('一般项目', 300), ('病史特点', 300), ('诊疗经过', 299), ('出院情况', 299)]

# 3. Randomize Train & Test

---

TODO:

Use `Cross Validation`

---

In [12]:
section_num = 4
cctTrain = []
cctTest  = []

for t in CCTDict:
    typeCCT = CCTDict[t]
    random.seed(10)
    l = random.sample(range(len(typeCCT)), int(len(typeCCT)/section_num))
    l.sort()
    cctTypeTrain = [typeCCT[idx] for idx in l]
    cctTrain.append(cctTypeTrain)
    cctTypeTest  = [typeCCT[idx] for idx in range(len(typeCCT)) if idx not in l]
    cctTest.append(cctTypeTest)

In [4]:
print(len(cctTest)); print([len(i) for i in cctTest]) # each are of 4 cct types
print(len(cctTrain));print([len(i) for i in cctTrain])


4
[225, 225, 225, 225]
4
[75, 75, 74, 74]


---

Log:

2018-06-18

Design `Cross Validation`



In [41]:
import numpy as np
from pprint import pprint


cross_num = 10
length = 205


length_pad = (int(length/10) + 1 ) * 10
print(length_pad)
new_list = list(range(length)) + [-1] * (length_pad - length)
print(new_list)



cross_index = np.array(random.sample(new_list, length_pad))
#cross_index.sort()
cross_index = cross_index.reshape((cross_num, -1))

print(cross_index.shape)
pprint(cross_index)

210
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, -1, -1, -1, -1, -1]
(10, 21)
array([[ 38,  68,  78,  34,  28, 158, 129, 109, 143

In [62]:
def getCrossValidationIdx(length, cross_num, pad = -1):
    
    length_pad = (int(length/cross_num) + 1 ) * cross_num
    #print(length_pad)
    new_list = list(range(length)) + [pad] * (length_pad - length)
    #print(new_list)

    cross_index = np.array(random.sample(new_list, length_pad))
    #cross_index.sort()
    return cross_index.reshape((cross_num, -1))

In [68]:
cross_num = 4
length =309
pad = -1

cross_index = getCrossValidationIdx(length, cross_num, pad = pad)
pprint(cross_index)
pprint(cross_index[1,])

array([[187,  32, 164, 267, 245,  47, 297,  69, 302, 157,  66,  79, 106,
        181, 196,  24,  18, 172, 303, 201, 117,  85,  40,  -1, 177, 212,
         30,  91, 291, 202, 169, 265, 254, 237,  28, 108, 188, 141, 104,
         43,  98, 229, 295,  37, 115, 150, 173, 127,   1, 232,  50, 144,
        134, 225,  17, 203, 301,  82, 210,  95, 123, 130, 189, 293, 224,
          4, 197, 176, 235, 140, 168, 240,  -1, 179, 217, 294,  20, 158],
       [ 35, 218, 131, 268, 166, 275,  62, 285,  73, 139,  13, 111, 243,
        284, 296,  16, 286, 292, 230, 162, 205,  65, 283,   5, 186,  63,
        167, 239, 199,  75, 133,  19, 178, 269, 272, 138, 149, 236,  26,
        214, 129, 280, 161, 211, 270, 180,  84,  94,  86, 110,  77, 135,
         90, 274, 163,  31,  38, 216, 233, 185, 213,  39, 276,  27, 145,
         70, 122, 248, 242, 290,  60, 257, 307, 154, 118,  10, 266, 107],
       [ 48, 200, 305, 258, 148,  52,  61, 204,  54, 304,  46, 259, 113,
        174, 228, 152, 194, 170,  45, 247,   6, 1

**Given Cross_Index, get Train and Test for each Loop i**



In [69]:
seed = 10

cross_idx = 2
cross_num = 4
pad = -1
    
    
random.seed(seed)

cctTrain = []
cctTest  = []
for t in CCTDict:
    typeCCT = CCTDict[t]
    cross_index = getCrossValidationIdx(len(typeCCT), cross_num, pad = pad)
    test_index = cross_index[cross_idx, ]
    test_index = [idx for idx in test_index if idx is not pad] # get rid of pad
    #l = random.sample(range(len(typeCCT)), int(len(typeCCT)/section_num))
    #l.sort()
    cctTypeTest = [typeCCT[idx] for idx in test_index]
    cctTest.append(cctTypeTest)
    cctTypeTrain  = [typeCCT[idx] for idx in range(len(typeCCT)) if idx not in test_index]
    cctTrain.append(cctTypeTrain)
    
print(len(cctTest)); print([len(i) for i in cctTest]) # each are of 4 cct types
print(len(cctTrain));print([len(i) for i in cctTrain])


4
[76, 76, 75, 75]
4
[224, 224, 224, 225]


In [74]:
def loadData(pklDictPath, batch,  cross_num,  
             seed = 10,  
             cross_validation = False, 
             cross_idx = 0, ):

    random.seed(seed)

    with open(pklDictPath, 'rb') as handle:
        cctDict_R = pickle.load(handle)
    
    cctTrain = []
    cctTest  = []

    if not cross_validation:

        for t in batch['dataInput']['filenames']:
            typeCCT = CCTDict[t]
            random.seed(10)
            l = random.sample(range(len(typeCCT)), int(len(typeCCT)/cross_num))
            l.sort()
            cctTypeTrain = [typeCCT[idx] for idx in l]
            cctTrain.append(cctTypeTrain)
            cctTypeTest  = [typeCCT[idx] for idx in range(len(typeCCT)) if idx not in l]
            cctTest.append(cctTypeTest)
    else:
        
        for t in  batch['dataInput']['filenames']:
            typeCCT = cctDict_R[t]
            cross_index = getCrossValidationIdx(len(typeCCT), cross_num, pad = pad)
            test_index = cross_index[cross_idx, ]
            test_index = [idx for idx in test_index if idx is not pad] # get rid of pad
            #l = random.sample(range(len(typeCCT)), int(len(typeCCT)/section_num))
            #l.sort()
            cctTypeTest = [typeCCT[idx] for idx in test_index]
            cctTest.append(cctTypeTest)
            cctTypeTrain  = [typeCCT[idx] for idx in range(len(typeCCT)) if idx not in test_index]
            cctTrain.append(cctTypeTrain)
        
    return cctTrain, cctTest

In [75]:
from dataset import batch_CCKS, batch_LUOHU, batch_LH_M, batch_LH_A, generateOriAn

pklDictPath = CCT_DictPath
cross_num = 4
batch = batch_CCKS
seed = 10

cctTrain, cctTest = loadData(pklDictPath, cross_num, batch, seed,  
                             cross_validation = False, 
                             cross_idx = 0)


print(len(cctTest)); print([len(i) for i in cctTest]) # each are of 4 cct types
print(len(cctTrain));print([len(i) for i in cctTrain])


4
[225, 225, 225, 225]
4
[75, 75, 74, 74]


In [76]:

pklDictPath = CCT_DictPath
cross_num = 4
batch = batch_CCKS
seed = 10

cctTrain, cctTest = loadData(pklDictPath, cross_num, batch, seed,  
                             cross_validation = True, 
                             cross_idx = 0)


print(len(cctTest)); print([len(i) for i in cctTest]) # each are of 4 cct types
print(len(cctTrain));print([len(i) for i in cctTrain])


4
[76, 76, 75, 75]
4
[224, 225, 225, 224]


In [77]:
pklDictPath = CCT_DictPath
cross_num = 10
batch = batch_CCKS
seed = 10

cctTrain, cctTest = loadData(pklDictPath, cross_num, batch, seed,  
                             cross_validation = True, 
                             cross_idx = 0)


print(len(cctTest)); print([len(i) for i in cctTest]) # each are of 4 cct types
print(len(cctTrain));print([len(i) for i in cctTrain])


4
[31, 31, 30, 30]
4
[269, 270, 269, 269]


# 4. Train Data

---

**Start Here**

Here we use `1ap` model.

`attr_cols = ['_atom', 'POSTag']`

`tag_cols  = ['ETag']`

In [5]:
import numpy as np
import pandas as pd

attr_cols = ['_atom', 'POSTag']
tag_cols  = ['ETag']

# open it, if you want to get the whole thing

DFtrain = pd.DataFrame()
for t in cctTrain:
    for cct in t:
        df = cct.dfFormat.copy()
        df = df.dropna()
        df.loc[len(df)] = np.NaN     ## Trick Here
        df = df[attr_cols + tag_cols]
        DFtrain = DFtrain.append(df) ## Trick Here

print(DFtrain.shape)
DFtrain.head(10)        

(63136, 3)


,_atom,POSTag,ETag
0,痛,a-B,Sy-B
1,1,m-B,O
2,0,m-I,O
3,小,n-B,O
4,时,n-I,O
5,。,x-B,O
6,NaN,NaN,NaN
0,伤,v-B,O
1,后,f-B,O
2,疼,n-B,Sy-B


In [6]:
print(df.shape)
df.head()

(119, 3)


,_atom,POSTag,ETag
0,现,tg-B,O
1,患,n-B,O
2,者,n-I,O
3,一,a-B,O
4,般,a-I,O


# 5. Add Word2Vec

---


In [7]:
cct.dfFormat.head()

,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag
0,现,CHN,OTHER,王,tg-B,O,O
1,患,CHN,OTHER,心,n-B,O,O
2,者,CHN,OTHER,耂,n-I,O,O
3,一,CHN,OTHER,一,a-B,O,O
4,般,CHN,OTHER,舟,a-I,O,O


In [8]:
from vector import cct2VecDF

DIM = 50

df_vect = cct2VecDF(cct, dim = DIM)
print(df_vect.shape)
df_vect.head()

(118, 51)


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,现,-0.143170,-0.462773,-0.155756,-0.207766,-0.267619,0.027132,0.074602,0.361419,0.161064,...,-0.084455,-0.029046,-0.439378,-0.135591,-0.424844,-0.451665,0.117317,-0.008237,-0.229197,-0.104767
1,患,-0.297559,-0.292324,-0.004288,-0.167847,-0.007375,0.200336,0.105082,-0.156831,0.258051,...,0.208579,-0.026332,-0.296286,0.185700,-0.061464,-0.405424,0.015128,-0.274715,-0.387244,-0.029484
2,者,-0.257371,-0.249603,0.057283,-0.157507,-0.066411,0.102987,0.086166,-0.019617,0.133194,...,0.198930,-0.025447,-0.237868,0.232395,-0.085046,-0.369719,-0.017688,-0.267530,-0.359067,-0.090966
3,一,-0.207889,0.095896,-0.185780,-0.187293,-0.384603,0.411974,0.052494,0.137288,-0.349163,...,0.092422,0.100060,0.202488,0.223583,0.157341,0.032043,-0.145194,-0.300491,-0.497292,-0.624846
4,般,-0.331893,0.172055,-0.472268,-0.371457,-0.697807,0.009928,0.042815,-0.180445,-0.406518,...,-0.087808,0.037746,0.566841,0.224230,0.298406,-0.005423,-0.761420,-0.113320,-0.774309,-0.964647


In [9]:
df_total = pd.concat([df, df_vect], axis=1)
# df_total

DIM = 50
attr_cols = ['_atom']
vect_cols = list(range(1, DIM + 1))

tag_cols = ['ETag']

df_total[attr_cols+vect_cols+ tag_cols].head()

,_atom,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,ETag
0,现,-0.143170,-0.462773,-0.155756,-0.207766,-0.267619,0.027132,0.074602,0.361419,0.161064,...,-0.029046,-0.439378,-0.135591,-0.424844,-0.451665,0.117317,-0.008237,-0.229197,-0.104767,O
1,患,-0.297559,-0.292324,-0.004288,-0.167847,-0.007375,0.200336,0.105082,-0.156831,0.258051,...,-0.026332,-0.296286,0.185700,-0.061464,-0.405424,0.015128,-0.274715,-0.387244,-0.029484,O
2,者,-0.257371,-0.249603,0.057283,-0.157507,-0.066411,0.102987,0.086166,-0.019617,0.133194,...,-0.025447,-0.237868,0.232395,-0.085046,-0.369719,-0.017688,-0.267530,-0.359067,-0.090966,O
3,一,-0.207889,0.095896,-0.185780,-0.187293,-0.384603,0.411974,0.052494,0.137288,-0.349163,...,0.100060,0.202488,0.223583,0.157341,0.032043,-0.145194,-0.300491,-0.497292,-0.624846,O
4,般,-0.331893,0.172055,-0.472268,-0.371457,-0.697807,0.009928,0.042815,-0.180445,-0.406518,...,0.037746,0.566841,0.224230,0.298406,-0.005423,-0.761420,-0.113320,-0.774309,-0.964647,O


---

# 6 Final Version of `getTrainData`

Update: 2018/03/14

In [10]:
import time

def getTrainData(cctTrain, attr_cols = None, tag_cols = None,  
                 Path = None, 
                 Vector = False, vect_cols = None, Vdim = None):

    DFtrain = pd.DataFrame()
    for t in cctTrain:
        for cct in t:
            df = cct.dfFormat.copy()
            df = df.dropna()
            df.loc[len(df)] = np.NaN     ## Trick Here
            df = df[attr_cols + tag_cols]
            
            if Vector == True and vect_cols:
                df_vec = cct2VecDF(cct, dim = Vdim)
                df_vec = df_vec.dropna()
                df_vec.loc[len(df_vec)] = np.NaN
                assert len(df_vec) == len(df)
                df = pd.concat([df, df_vec], axis = 1)
                df = df[attr_cols + vect_cols + tag_cols]
                #print(df)
                
            DFtrain = DFtrain.append(df) ## Trick Here
    DFtrain = DFtrain.reset_index(drop=True)
    #DFtrain = DFtrain[cols]
    DFtrain.to_csv(Path, sep = '\t', encoding = 'utf=8', header = False, index = False )
    return DFtrain

In [11]:
DIM = 50

trainDataPara = {'vect_cols': list(range(1,DIM + 1)),
                 'Vector': True,
                 'Vdim': 50,
                 'attr_cols': ['_atom'],
                 'tag_cols' : ['ETag'],
                 'Path':'demo/train.txt'}

df_final = getTrainData([cctTrain[0][:10]], **trainDataPara)
print(df_final.shape)
df_final.head(10)

(495, 52)


,_atom,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,ETag
0,痛,0.163473,0.377304,-0.117544,0.088628,-0.103927,-0.263283,-0.233817,0.281086,-0.201166,...,-0.325955,-0.221439,0.145688,-0.501243,0.119017,-0.217983,-0.352906,-0.214400,-0.163277,Sy-B
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,O
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,O
3,小,0.163001,0.006206,-0.576872,0.113557,-0.227850,0.010900,-0.171475,-0.120443,-0.246896,...,-0.001984,0.105811,-0.061227,0.227538,0.308686,-0.507730,-0.149098,-0.343513,-0.084501,O
4,时,-0.139881,-0.306468,-0.348440,0.109640,-0.279528,-0.401167,0.190646,0.126747,-0.110307,...,0.116581,-0.515175,-0.242006,0.451827,0.025621,-0.251690,-0.289996,-0.142036,-0.385012,O
5,。,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,O
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,伤,-0.315089,0.336364,-0.101161,-0.262826,0.221417,0.521034,-0.226776,-0.012057,0.216877,...,-0.411392,0.127450,-0.169097,0.312362,-0.083843,-0.007500,-0.446096,-0.817941,0.283945,O
8,后,-0.104686,-0.071469,-0.023803,-0.145819,-0.080070,0.465533,0.016220,-0.098209,-0.112330,...,0.117080,-0.300014,0.268646,0.183839,-0.199503,0.074922,-0.013016,-0.202738,0.028909,O
9,疼,-0.063051,0.245123,-0.161457,0.188132,-0.184917,-0.140965,-0.060787,0.116404,-0.192944,...,-0.440119,-0.278272,0.193039,-0.047440,-0.083968,0.072703,-0.156498,-0.311215,-0.436355,Sy-B


# 7 Get Test Data

---

**A Start**

In [12]:
DFtest = pd.DataFrame()
for t in cctTest:
    for cct in t:
        df = cct.dfFormat.copy()
        df = df.dropna()
        df.loc[len(df)] = np.NaN     ## Trick Here
        DFtest = DFtest.append(df)   ## Trick Here

DFtest = DFtest.reset_index(drop=True)
print(DFtest.shape)
DFtest.head(10)

(199919, 7)


,_atom,basicTag,dictTag,radical,POSTag,RTag,ETag
0,女,CHN,OTHER,女,n-B,O,O
1,性,CHN,OTHER,忄,n-I,O,O
2,",",PUNC,OTHER,-,x-B,O,O
3,8,NUM,OTHER,-,m-B,O,O
4,8,NUM,OTHER,-,m-I,O,O
5,岁,CHN,OTHER,山,m-B,O,O
6,",",PUNC,OTHER,-,x-B,O,O
7,农,CHN,OTHER,冖,n-B,O,O
8,民,CHN,OTHER,氏,n-I,O,O
9,",",PUNC,OTHER,-,x-B,O,O


---

**Whole Processing**

In [13]:
def getTestData(cctTest, attr_cols = None,  
                Path = None,
                Vector = False,
                Vdim = None,
                vect_cols = None):

    DFtest = pd.DataFrame()
    for t in cctTest:
        for cct in t:
            cct.outputCols = attr_cols
            df = cct.dfFormat.copy()
            df = df.dropna()
            df.loc[len(df)] = np.NaN    ## Trick Here
            df = df[attr_cols]
            
            if Vector == True and vect_cols:
                df_vec = cct2VecDF(cct, dim = Vdim)
                df_vec = df_vec.dropna()
                df_vec.loc[len(df_vec)] = np.NaN
                assert len(df_vec) == len(df)
                df = pd.concat([df, df_vec], axis = 1)
                # vect_cols = list(range(1, 50)) # TODO, to deal with DIM, change it later
                df = df[attr_cols + vect_cols ]

            DFtest = DFtest.append(df)   ## Trick Here

    DFtest = DFtest.reset_index(drop=True)
    DFtest.to_csv(Path, sep = '\t', encoding = 'utf=8', 
                  header = False, index = False )
    return DFtest

In [14]:
DIM = 50

testDataPara = {'attr_cols': ['_atom'],
                'vect_cols': list(range(1,DIM + 1)),
                'Vector': True,
                'Vdim': 50,
                'Path':'demo/test.txt'}

df_final = getTestData([cctTest[0][:10]], **testDataPara)

print(df_final.shape)
df_final.head(10)

(330, 51)


,_atom,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,女,-0.384153,-0.449407,-0.000619,-0.329871,-0.013730,-0.254459,-0.055858,-0.750129,0.170088,...,0.174711,-0.170360,0.143862,0.320878,-0.055447,-0.493701,-0.032973,0.000014,-0.405085,-0.329773
1,性,-0.095533,-0.067222,-0.140703,0.128773,0.218588,-0.178781,0.253661,0.040975,-0.008195,...,0.002214,-0.246365,-0.330390,0.367284,-0.526000,0.076973,-0.060376,-0.101708,0.009696,0.103741
2,",",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,岁,-0.310619,-1.014080,-0.069569,-0.461891,0.032117,-0.219336,-0.021506,-0.726061,0.183611,...,0.757968,-0.046885,-0.004123,-0.266823,-0.105086,-0.127436,0.045561,0.255756,-0.551634,-0.256877
6,",",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,农,-1.301767,-0.757325,0.818786,-0.254610,0.197441,-0.429172,-1.036751,-0.532633,0.613818,...,-1.463534,-0.675300,0.909776,0.022699,0.378455,0.355505,-1.028394,-0.159682,-1.634793,1.307894
8,民,-0.193231,-0.132735,-0.255224,-1.742414,-0.248409,0.135128,-1.003910,0.298898,-0.505406,...,-0.830977,0.429896,-0.097209,-0.181830,0.950165,0.080404,-0.562398,0.863556,0.573977,0.544305
9,",",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 8. Test2 Data

In [26]:
Test2 = pd.read_csv('demo/LoadData/vect_result_R.txt', sep = '\t', header = None, skip_blank_lines=False)
print(Test2.shape)
Test2.head(10)

(508, 52)


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,女,0.010178,-0.146067,-0.086911,-0.083372,0.478019,-0.622843,1.134277,-0.662709,-0.061658,...,0.431023,0.488524,-0.448062,0.345038,0.462508,-0.324386,-0.076126,-0.049036,0.426018,O
1,性,0.275105,0.451722,-0.324697,-0.097799,-0.270617,-0.340291,0.231919,-0.615251,0.345565,...,0.103549,0.384304,-0.268350,0.299864,0.582433,-0.267846,0.066516,-0.160935,-0.263605,O
2,",",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,O
3,8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,O
4,8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,O
5,岁,0.155483,-0.052854,0.272934,0.366036,0.453302,-0.273168,0.833754,-0.237104,0.063369,...,0.814165,0.220632,0.446104,0.964465,-0.013553,-0.468324,0.267131,0.466923,0.366577,O
6,",",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,O
7,农,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,O
8,民,1.594264,0.569491,0.575772,-0.444499,1.046615,1.024563,-0.811281,-0.021639,-1.114609,...,-0.424888,-0.449514,0.413471,0.326465,0.204650,0.099314,-0.661626,-0.492835,-0.478434,O
9,",",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,O


In [16]:
def getTest2Data(TempResultPath,
                 Test2DataPath):
    
    Test2Data = pd.read_csv(TempResultPath, sep = '\t', header = None, skip_blank_lines=False)
    Test2Data.to_csv(Test2DataPath, sep = '\t', encoding = 'utf=8', header = False, index = False )

# 9. Eval

---

We don't have the Predicted `result_R` and `result_E`, yet.

---

In [27]:
'''

from evals import evalPerform

p = 'demo/LoadData/vect_result_E.txt'
cols = attr_cols + vect_cols
evalPerform([cctTest[0][:10]], p, 'LearnedETag', cols)

'''

"\n\nfrom evals import evalPerform\n\np = 'demo/LoadData/vect_result_E.txt'\ncols = attr_cols + vect_cols\nevalPerform([cctTest[0][:10]], p, 'LearnedETag', cols)\n\n"